In [ ]:
import pandas as pd

In [ ]:
file = "data/tblTokensNew_180301.xlsx"

In [ ]:
df = pd.read_excel(file)

In [ ]:
# create necessary concept-schemes
ecce_pos,_ = SkosConceptScheme.objects.get_or_create(dc_title="ecce-pos")
lemma_pos,_ = SkosConceptScheme.objects.get_or_create(dc_title="lemma-pos")

In [ ]:
pos = set(df['PoS'])

In [ ]:
for x in pos:
    temp_variable,_ = SkosConcept.objects.get_or_create(pref_label=x)
    temp_variable.save()
    temp_variable.scheme.set([ecce_pos])

In [ ]:
# ceate objects
missing_clusters = []
for i, row in df.iterrows():
    temp_object,_ = Token.objects.get_or_create(legacy_id=row['ID'])
    temp_object.spelling=row['spelling']
    temp_object.left_context=row['leftcontext']
    temp_object.plain_word=row['plainword']
    temp_object.comments=row['comments']
    temp_object.right_context=row['rightcontext']
    temp_object.file=row['file']
    temp_object.medial_suffix=row['medialsuffix']
    temp_object.final_suffix=row['finalsuffix']
    temp_object.weight = row['weight']
    temp_object.weight_norm = row['weightPerMilPerSemi']
    temp_object.weight_not_norm = row['awPerMilPerSemi']
    try:
        temp_pos = SkosConcept.objects.get(pref_label=row['PoS'])
        temp_object.pos=temp_pos
        temp_object.save()
    except:
        pass
    try:
        temp_fk = TokenLabel.objects.get(label=row['label'])
        temp_object.label=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = Text.objects.get(text=row['textsource'])
        temp_object.text_source=temp_fk
        temp_object.save()
    except:
        pass
    try:
        print(row['cluster'])
        temp_fk = Cluster.objects.get(consonant=row['cluster'])
        temp_object.cluster=temp_fk
        temp_object.save()
    except:
        print(row['cluster'])
        missing_clusters.append(row['cluster'])
        pass
    try:
        temp_fk = SchwaPresent.objects.get(spelling=row['spelling2'])
        temp_object.spelling2=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = OnSet.objects.get(rightonset=row['rightonset'])
        temp_object.rightonset=temp_fk
        temp_object.save()
    except:
        pass
    temp_lempos, _ = SkosConcept.objects.get_or_create(
        pref_label=row['lemmaPoS'],
        scheme=lemma_pos
    )
    new_lemma, _ = Lemma.objects.get_or_create(
        name=row['lemma'], pos=temp_lempos
    )
    temp_object.lemma = new_lemma
    temp_object.save()

In [ ]:
#Token.objects.all().delete()